# Занятие 3. Итераторы, генераторы и context manager


## Декораторы

Декоратор — это объект, который расширяет возможности функции, не меняя её исходный кода.

Принцип работы декоратора:
1. принимает функцию как аргумент
2. объявляет новую функцию, которая расширяет функцию-аргумент
3. возвращает новую функцию в качестве объекта.


In [ ]:
def counter(f):
    total = 0

    # Объявляем функцию, которая расширяет функционал f
    def decorated(*args, **kwargs):
        # Переменная total объявлена нелокальной для доступа из внутренней функции
        nonlocal total
        total += 1
        # Возвращаем значение исходной функции и дополнительно total
        return f(*args, **kwargs), total
    # Возвращаем новую функцию как объект
    return decorated


@counter
def hello(name):
    return f"Привет, {name}!"


print(hello("Студент_1"))
print(hello("Студент_404"))

In [ ]:
import time
import random

def timer(f):
    # Объявляем функцию, которая расширяет функционал f
    def decorated(*args, **kwargs):
        # Переменная total объявлена нелокальной для доступа из внутренней функции
        start = time.time()
        # Возвращаем значение исходной функции и дополнительно total
        executed = f(*args, **kwargs)
        total_time = time.time() - start
        return executed, total_time
    # Возвращаем новую функцию как объект
    return decorated


@timer
def hello(name):
    time.sleep(random.random())
    return f"Привет, {name}!"


print(hello("Студент_1"))
print(hello("Студент_404"))

### Фабрика декораторов

In [ ]:
from time import perf_counter, sleep
import random

def decorator_factory(loops_num):
    def decorating(fn):
        def inner(*args, **kwargs):   
            total_elapsed = 0
            for i in range(loops_num):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += end - start
            avg_run_time = total_elapsed/loops_num
            print('result is', result)    
            print('num of loops is', loops_num)
            print('avg time elapsed', avg_run_time)
        return inner
    return decorating

@decorator_factory(3)
def hello(name):
    sleep(random.random())
    return f"Привет, {name}!"

hello("Студент_418")

In [ ]:
class Decorator_Factory_Class:
    def __init__(self, num_loops):
        self.num_loops = num_loops
    def __call__(self, fn):
          def inner(num):   
            total_elapsed = 0
            for i in range(self.num_loops):
                start = perf_counter()
                result = fn(num)
                end = perf_counter()
                total_elapsed += end - start
            avg_run_time = total_elapsed/self.num_loops
            print('num of loops is', self.num_loops)
            return result
          return inner


@decorator_factory(3)
def hello(name):
    sleep(random.random())
    return f"Привет, {name}!"

hello("Студент_418")

### Декораторы классов

In [ ]:
class University:
    def __init__(self, name):
        self.name = name

    @staticmethod
    def say_greeting():
        print("Welcome to our university!")

University.say_greeting()

In [ ]:
class University:
    _subjects = ["Math", "English", "Physics"]
    def __init__(self, name):
        self.name = name

    @classmethod
    def reduce_subjects(cls):
        cls._subjects.pop()

    def get_subjects(self):
        return self._subjects

misis = University("MISIS")
print(misis.get_subjects())
University.reduce_subjects()
print(misis.get_subjects())

In [ ]:
class Rectangle:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    @property
    def area(self):
        return self.a * self.b

rect = Rectangle(5, 6)
print(rect.area)

## Итераторы

Итератор (iterator) - это объект, который возвращает свои элементы по одному за раз.

С точки зрения Python - это любой объект, у которого определены магические методы `__next__` и `__iter__`.

`__next__` - этот метод возвращает следующий элемент, если он есть, или возвращает исключение StopIteration, когда элементы закончились.

`__iter__` - этот метод просто возвращает сам итератор.

In [ ]:
langs = ["Python", "Golang", "C++"]
iter(langs)

In [ ]:
iter_langs = iter(["Python", "Golang", "C++"])
print(next(iter_langs))
list(iter_langs)

In [ ]:
langs = ["Python", "Golang", "C++"]
for lang in iter(langs): # == for lang in langs
    print(lang)

* Зачем нужны итераторы? 

In [ ]:
file = open('example.txt')
file.__next__()

In [ ]:
for line in file:
    print(line.rstrip())

### Генераторы

Каждый генератор - это итератор, но не наоборот (== генератор это подтип итератора)

In [ ]:
from typing import Iterator
from typing import Generator

print(f'issubclass(Generator, Iterator) = {issubclass(Generator, Iterator)}')
print(f'issubclass(Iterator, Generator) = {issubclass(Iterator, Generator)}')

#### Генераторное выражение

In [ ]:
genexpr = (int(input()) ** 2 for x in range(3))  # ожидания ввода нет

In [ ]:
next(genexpr)

#### Функция-генератор

In [ ]:
import time


def fib(n):  # числа Фиббоначи
    n_1, n_2 = 1, 1
    for i in range(n):
        print("Stop execution")
        yield n_1
        print("Continue execution")
        n_1, n_2 = n_2, n_1 + n_2

for x in fib(5):
    print("Current number:", x)
    time.sleep(2)

## Context manager

>«Контекстные менеджеры в Python — это удивительный механизм, который позволяет гарантировать корректное управление ресурсами и обеспечивать безопасное выполнение кода.» — Гвидо ван Россум, создатель языка программирования Python.

### Работа с файлами

In [ ]:
file = open("example.txt", "r")
try:
    # Действия с файлом
    content = file.read()
    print(content)
finally:
    file.close()

In [ ]:
with open("example.txt", "r") as file:
    content = file.read()
    print(content)
# --> file.close()

### Работа с бд

In [ ]:
import sqlite3

# Пример работы с SQLite базой данных
with sqlite3.connect('example.db') as conn:
    cursor = conn.cursor()
    # Выполнение операций с базой данных
    cursor.execute('SELECT * FROM users')
    result = cursor.fetchall()
    print(result)

### Свой контекстный менеджер

Контекстный менеджер в Python должен содержать методы `__enter__` и `__exit__`.

Метод `__enter__` выполняется перед выполнением блока кода внутри оператора with. Он может выполнять какие-либо подготовительные действия или возвращать значение, которое будет связано с переменной после ключевого слова as.

Метод `__exit__` вызывается после завершения выполнения блока кода with. Он используется для выполнения завершающих действий, таких как освобождение ресурсов, обработка исключений или выполнение финализирующих операций.

In [14]:
class ContextManager(): 
    def __init__(self): 
        print('init method called') 
          
    def __enter__(self): 
        print('enter method called') 
        return self
      
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        print('exit method called') 
  
  
with ContextManager() as manager: 
    print('with statement block') 

init method called
enter method called
with statement block
exit method called


In [ ]:
class FileManager(): 
    def __init__(self, filename, mode): 
        self.filename = filename 
        self.mode = mode 
        self.file = None
          
    def __enter__(self): 
        self.file = open(self.filename, self.mode) 
        return self.file
      
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        self.file.close() 
  
# загрузка файла
with FileManager('example.txt', 'r') as f: 
    print(f.encoding)
  
print(f.closed) 

In [17]:
from contextlib import contextmanager

@contextmanager
def context_manager():
    # Внутри вызова __enter__
    print("enter method called")
    try:
        yield
    finally:
        # Внутри вызова __exit__
        print("exit method called")

with context_manager() as manager:
    print("with statement block")

enter method called
with statement block
exit method called


In [20]:
import asyncio
 
# определение асинхронного менеджера контекста
class AsyncContextManager:
    # вход в асинхронный менеджер контекста
    async def __aenter__(self):
        # вывод сообщения
        print('>entering the context manager')
        # блокировка на некоторое время
        await asyncio.sleep(0.5)
 
    # выход из асинхронного менеджера контекста
    async def __aexit__(self, exc_type, exc, tb):
        # вывод сообщения
        print('>exiting the context manager')
        # блокировка на некоторое время
        await asyncio.sleep(0.5)
 
# определение простой корутины
async def custom_coroutine():
    # создание и использование асинхронного менеджера контекста
    async with AsyncContextManager() as manager:
        # вывод результирующего сообщения
        print(f'within the manager')
 
# запуск asyncio-программы
await custom_coroutine()

>entering the context manager
within the manager
>exiting the context manager


## Доп. материалы

[Статья про декораторы и генераторы от Яндекса](https://academy.yandex.ru/handbook/python/article/rekursiya-dekoratory-generatory)

[Некоторые хитрости при работе с итераторами](https://habr.com/ru/articles/488112/)

[Подробнее про context manager и его применения](https://realpython.com/python-with-statement/)

Для продвинутых: 

[Интересная статья про асинхронность: контекстные менеджеры и не только](https://habr.com/ru/companies/wunderfund/articles/711012/)